In [1]:
import pandas as pd

## Input: A single excel file which concatenated all blocks (112) halfhourly data, each row representing invidual block & LCLid halfhourly data (1 row represents one unique LCLid in one block)
## Output: A single excel file, energy consumption aggregated by date and hour, LCLid aggregated as unique count per hour, block id aggregated as unique block per hour.

In [2]:
destination_path = 'C:/Users/chiny/cjwen/ITI110/'


In [3]:
df = pd.read_csv(destination_path + "Raw_data/concatenated_halfhourly_dataset.csv")

In [4]:
df

,LCLid,tstp,energy(kWh/hh),block_id
0,MAC000002,2012-10-12 00:30:00.0000000,0,block_0
1,MAC000002,2012-10-12 01:00:00.0000000,0,block_0
2,MAC000002,2012-10-12 01:30:00.0000000,0,block_0
3,MAC000002,2012-10-12 02:00:00.0000000,0,block_0
4,MAC000002,2012-10-12 02:30:00.0000000,0,block_0
...,...,...,...,...
167817016,MAC005019,2014-02-27 22:00:00.0000000,0.129,block_99
167817017,MAC005019,2014-02-27 22:30:00.0000000,0.095,block_99
167817018,MAC005019,2014-02-27 23:00:00.0000000,0.061,block_99
167817019,MAC005019,2014-02-27 23:30:00.0000000,0.054,block_99


In [5]:
# Remove outlier meters
to_remove = ['MAC000004', 
 'MAC000037', 
 'MAC000197', 
 'MAC000325', 
 'MAC000399', 
 'MAC000895', 
 'MAC001309', 
 'MAC001976', 
 'MAC002050', 
 'MAC002110', 
 'MAC002136', 
 'MAC002564', 
 'MAC002594', 
 'MAC002858', 
 'MAC002873', 
 'MAC002976', 
 'MAC003156', 
 'MAC003327', 
 'MAC003627', 
 'MAC004067', 
 'MAC004672', 
 'MAC004731', 
 'MAC004735', 
 'MAC005033', 
 'MAC005069', 
 'MAC005313']

In [6]:
df = df[~df["LCLid"].isin(to_remove)] #remove outlier meters

In [9]:
# df[df["LCLid"]=="MAC000004"]

In [10]:
#Format data
df['energy(kWh/hh)'] = pd.to_numeric(df['energy(kWh/hh)'], errors='coerce')
df['tstp'] = pd.to_datetime(df['tstp'])

C:\Users\chiny\AppData\Local\Temp\ipykernel_91104\1440308734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['energy(kWh/hh)'] = pd.to_numeric(df['energy(kWh/hh)'], errors='coerce')
C:\Users\chiny\AppData\Local\Temp\ipykernel_91104\1440308734.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tstp'] = pd.to_datetime(df['tstp'])


In [11]:
#extract time features
df["date"] = df["tstp"].dt.date
df["month"]= df["tstp"].dt.month
df["hour"]= df["tstp"].dt.hour

C:\Users\chiny\AppData\Local\Temp\ipykernel_91104\2912657088.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date"] = df["tstp"].dt.date
C:\Users\chiny\AppData\Local\Temp\ipykernel_91104\2912657088.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["month"]= df["tstp"].dt.month
C:\Users\chiny\AppData\Local\Temp\ipykernel_91104\2912657088.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167068985 entries, 0 to 167817020
Data columns (total 7 columns):
 #   Column          Dtype         
---  ------          -----         
 0   LCLid           object        
 1   tstp            datetime64[ns]
 2   energy(kWh/hh)  float64       
 3   block_id        object        
 4   date            object        
 5   month           int32         
 6   hour            int32         
dtypes: datetime64[ns](1), float64(1), int32(2), object(3)
memory usage: 8.7+ GB


In [14]:
agg_df = df.groupby(['date','hour']).agg({
            'date': 'max',
            'month':'max',
            'hour': 'max',
            'LCLid': lambda x: x.nunique(),
            'block_id': lambda y: y.nunique(),
            'energy(kWh/hh)': 'sum'
        }).reset_index(drop=True)

In [21]:
agg_df.rename(columns={"LCLid": "count_LCLid", "block_id": "count_block_id","energy(kWh/hh)":"sum_energy_by_hour"},inplace=True)

In [23]:
agg_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19864 entries, 0 to 19863
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                19864 non-null  object 
 1   month               19864 non-null  int32  
 2   hour                19864 non-null  int32  
 3   count_LCLid         19864 non-null  int64  
 4   count_block_id      19864 non-null  int64  
 5   sum_energy_by_hour  19864 non-null  float64
dtypes: float64(1), int32(2), int64(2), object(1)
memory usage: 931.1+ KB


In [24]:
agg_df.isnull().sum()

date                  0
month                 0
hour                  0
count_LCLid           0
count_block_id        0
sum_energy_by_hour    0
dtype: int64

In [25]:
#create a timestamp column
agg_df["date"]= agg_df["date"].astype(str)
agg_df['hour'] = agg_df['hour'].apply(lambda x: f"{x:02d}")
agg_df['tstp'] = pd.to_datetime(agg_df['date'] + ' ' + agg_df['hour'] + ':00:00.0000000')

In [27]:
agg_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19864 entries, 0 to 19863
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                19864 non-null  object        
 1   month               19864 non-null  int32         
 2   hour                19864 non-null  object        
 3   count_LCLid         19864 non-null  int64         
 4   count_block_id      19864 non-null  int64         
 5   sum_energy_by_hour  19864 non-null  float64       
 6   tstp                19864 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2), object(2)
memory usage: 1.1+ MB


In [29]:
agg_df.groupby("date")["sum_energy_by_hour"].sum()

date
2011-11-23       90.385000
2011-11-24      213.412000
2011-11-25      303.993000
2011-11-26      420.976000
2011-11-27      444.883001
                  ...     
2014-02-24    51962.457004
2014-02-25    51386.567001
2014-02-26    50907.022994
2014-02-27    51647.284998
2014-02-28     1042.087000
Name: sum_energy_by_hour, Length: 829, dtype: float64

In [31]:
agg_df.to_csv("1.2 aggregated_hourly_dataset_without_block_id_02112024.csv",index=False)